## Libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
%matplotlib inline

## Importing the Whole Dataset

In [3]:
links = pd.read_csv('links.csv')

In [4]:
ratings = pd.read_csv('ratings.csv')

In [5]:
tags = pd.read_csv('tags.csv')

In [6]:
movies = pd.read_csv('movies.csv')

## User-Genre Matrix

#### All Genres in the Dataset

In [7]:
genres = []
for i in movies.index:
    genres.extend(movies['genres'][i].split('|'))
genres = list(set(genres))
genres.remove('(no genres listed)')

In [8]:
len(genres)

19

In [9]:
users = list(ratings['userId'].unique())

In [10]:
usergenre_matrix = pd.DataFrame(columns = genres, index = users)

In [11]:
movies_temp = movies.copy()
for i in range(9742):
    movies_temp['genres'].iloc[i] = (movies_temp['genres'].iloc[i]).split('|')
movies_temp = movies_temp.explode('genres')
movies_temp.index = np.arange(0,len(movies_temp))
movies_temp

C:\Users\Sibam\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,movieId,title,genres
0,1,Toy Story (1995),Adventure
1,1,Toy Story (1995),Animation
2,1,Toy Story (1995),Children
3,1,Toy Story (1995),Comedy
4,1,Toy Story (1995),Fantasy
...,...,...,...
22079,193583,No Game No Life: Zero (2017),Fantasy
22080,193585,Flint (2017),Drama
22081,193587,Bungo Stray Dogs: Dead Apple (2018),Action
22082,193587,Bungo Stray Dogs: Dead Apple (2018),Animation


In [12]:
result = pd.merge(movies_temp, ratings, left_on = 'movieId', right_on = 'movieId')
result

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure,1,4.0,964982703
1,1,Toy Story (1995),Adventure,5,4.0,847434962
2,1,Toy Story (1995),Adventure,7,4.5,1106635946
3,1,Toy Story (1995),Adventure,15,2.5,1510577970
4,1,Toy Story (1995),Adventure,17,4.5,1305696483
...,...,...,...,...,...,...
274475,193583,No Game No Life: Zero (2017),Fantasy,184,3.5,1537109545
274476,193585,Flint (2017),Drama,184,3.5,1537109805
274477,193587,Bungo Stray Dogs: Dead Apple (2018),Action,184,3.5,1537110021
274478,193587,Bungo Stray Dogs: Dead Apple (2018),Animation,184,3.5,1537110021


In [13]:
x = result.groupby(['userId', 'genres']).mean()
x = x.drop(columns = ['movieId', 'timestamp'])
x

rating
userId genres             
1      Action     4.322222
       Adventure  4.388235
       Animation  4.689655
       Children   4.547619
       Comedy     4.277108
...                    ...
610    Romance    3.731092
       Sci-Fi     3.659363
       Thriller   3.573529
       War        3.776596
       Western    3.742424

[10027 rows x 1 columns]

In [14]:
for i in genres:
    for j in users:
        try:
            usergenre_matrix[i][j] = x.loc[j, i][0]
        except:
            usergenre_matrix[i][j] = 0.0

In [15]:
usergenre_matrix

,Horror,Thriller,Musical,War,Animation,Drama,Action,Film-Noir,Children,Adventure,Fantasy,Western,Comedy,Mystery,Documentary,Crime,IMAX,Sci-Fi,Romance
1,3.47059,4.14545,4.68182,4.5,4.68966,4.52941,4.32222,5,4.54762,4.38824,4.29787,4.28571,4.27711,4.16667,0,4.35556,0,4.225,4.30769
2,3,3.7,0,4.5,0,3.88235,3.95455,0,0,4.16667,0,3.5,4,4,4.33333,3.8,3.75,3.875,4.5
3,4.6875,4.14286,0.5,0.5,0.5,0.75,3.57143,0,0.5,2.72727,3.375,0,1,5,0,0.5,0,4.2,0.5
4,4.25,3.55263,4,3.57143,4,3.48333,3.32,4,3.8,3.65517,3.68421,3.8,3.50962,3.47826,4,3.81481,3,2.83333,3.37931
5,3,3.55556,4.4,3.33333,4.33333,3.8,3.11111,0,4.11111,3.25,4.14286,3,3.46667,4,0,3.83333,3.66667,2.5,3.09091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,3.34615,3.52513,3.72727,3.79231,3.71429,3.78797,3.17881,3.8125,3.44898,3.5034,3.59794,3.41176,3.56532,3.79121,3.8,3.65414,3.0625,3.55696,3.74085
607,4.11429,4.11475,3.6,4.16667,3.33333,4.0122,3.72222,0,3.42105,3.46667,3.57143,4,3.32727,4.64706,0,3.81481,5,3.25,3.51724
608,3.31959,3.53668,2.75758,3.57895,3.11818,3.4375,3.33032,3.75,2.46023,3.22099,3,2.63636,2.73662,3.55072,3,3.61301,4,3.29641,2.88679
609,3.5,3.28571,0,3.5,3,3.36842,3.09091,0,3,3.2,3,4,3.28571,0,3,3.5,3,3,3.2


##### Normalize the User-Genre Matrix

In [16]:
usergenre_matrix=(usergenre_matrix-usergenre_matrix.min())/(usergenre_matrix.max()-usergenre_matrix.min())

In [17]:
usergenre_matrix

,Horror,Thriller,Musical,War,Animation,Drama,Action,Film-Noir,Children,Adventure,Fantasy,Western,Comedy,Mystery,Documentary,Crime,IMAX,Sci-Fi,Romance
1,0.694118,0.829091,0.936364,0.9,0.937931,0.889273,0.864444,1,0.909524,0.877647,0.859574,0.857143,0.855422,0.833333,0,0.871111,0,0.845,0.861538
2,0.6,0.74,0,0.9,0,0.737024,0.790909,0,0,0.833333,0,0.7,0.8,0.8,0.866667,0.76,0.75,0.775,0.9
3,0.9375,0.828571,0.1,0.1,0.1,0,0.714286,0,0.1,0.545455,0.675,0,0.2,1,0,0.1,0,0.84,0.1
4,0.85,0.710526,0.8,0.714286,0.8,0.643137,0.664,0.8,0.76,0.731034,0.736842,0.76,0.701923,0.695652,0.8,0.762963,0.6,0.566667,0.675862
5,0.6,0.711111,0.88,0.666667,0.866667,0.717647,0.622222,0,0.822222,0.65,0.828571,0.6,0.693333,0.8,0,0.766667,0.733333,0.5,0.618182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,0.669231,0.705025,0.745455,0.758462,0.742857,0.714815,0.635762,0.7625,0.689796,0.70068,0.719588,0.682353,0.713064,0.758242,0.76,0.730827,0.6125,0.711392,0.748169
607,0.822857,0.822951,0.72,0.833333,0.666667,0.767575,0.744444,0,0.684211,0.693333,0.714286,0.8,0.665455,0.929412,0,0.762963,1,0.65,0.703448
608,0.663918,0.707336,0.551515,0.715789,0.623636,0.632353,0.666065,0.75,0.492045,0.644199,0.6,0.527273,0.547324,0.710145,0.6,0.722603,0.8,0.659281,0.577358
609,0.7,0.657143,0,0.7,0.6,0.616099,0.618182,0,0.6,0.64,0.6,0.8,0.657143,0,0.6,0.7,0.6,0.6,0.64


## Movie Co-occurance Matrix (With Jaccard Similarity)

In [18]:
movie_matrix = pd.DataFrame(columns = movies.movieId, index = movies.movieId)

In [18]:
dff = ratings.copy()
dff =  ratings.groupby(['movieId'])['userId'].unique()
dff

movieId
1         [1, 5, 7, 15, 17, 18, 19, 21, 27, 31, 32, 33, ...
2         [6, 8, 18, 19, 20, 21, 27, 51, 62, 68, 82, 91,...
3         [1, 6, 19, 32, 42, 43, 44, 51, 58, 64, 68, 91,...
4                           [6, 14, 84, 162, 262, 411, 600]
5         [6, 31, 43, 45, 58, 66, 68, 84, 103, 107, 111,...
                                ...                        
193581                                                [184]
193583                                                [184]
193585                                                [184]
193587                                                [184]
193609                                                [331]
Name: userId, Length: 9724, dtype: object

In [19]:
'''for i in movie_matrix.index:
    for j in movie_matrix.index:
        try:
            print(i, j)
            movie_matrix[str(i)][j] = round(len(set(dff[i]).intersection(set(dff[j])))/len(set(dff[i]).union(set(dff[j]))), 2)
        except KeyError:
            movie_matrix[str(i)][j] = 0.0'''

'for i in movie_matrix.index:\n    for j in movie_matrix.index:\n        try:\n            print(i, j)\n            movie_matrix[str(i)][j] = round(len(set(dff[i]).intersection(set(dff[j])))/len(set(dff[i]).union(set(dff[j]))), 2)\n        except KeyError:\n            movie_matrix[str(i)][j] = 0.0'

In [20]:
movie_matrix = pd.read_csv('movie_matrix_final.csv')

In [21]:
movie_matrix.set_index(movies.movieId, inplace = True)

In [22]:
movie_matrix

,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
movieId,,,,,,,,,,,,,,,,,,,,,
1,1,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,0,1,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0,0,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,0,0,0,0,0,0,0,0,0,0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
193583,0,0,0,0,0,0,0,0,0,0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
193585,0,0,0,0,0,0,0,0,0,0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0


## Movie-Genre Matrix

In [19]:
movies_temp = movies.copy()

In [20]:
movies_temp = movies_temp.set_index('movieId')

In [21]:
movies_temp

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy
...,...,...
193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
193585,Flint (2017),Drama


In [22]:
movie_genre = pd.DataFrame(columns = genres, index = movies.movieId)

In [23]:
movie_genre[genres] = 0

In [24]:
for i in movies.movieId:
    for j in movies_temp['genres'][i].split('|'):
        try:
            movie_genre[j][i] = 1
        except KeyError:
            pass

In [25]:
movie_genre

,Horror,Thriller,Musical,War,Animation,Drama,Action,Film-Noir,Children,Adventure,Fantasy,Western,Comedy,Mystery,Documentary,Crime,IMAX,Sci-Fi,Romance
movieId,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,1,0,0,0,1,1,1,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
4,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1
5,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,0,0
193583,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0
193585,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


##### Predicting Top 10 Movies For each Users

In [26]:
user_movie_pred = {}
for i in users:
    print(i)
    array = (cosine_similarity(np.array(usergenre_matrix.loc[i]).reshape(1, 19), movie_genre))[0].argsort()[-10:][::-1]
    user_movie_pred[i] = list(movies.ix[array.tolist()].movieId)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


C:\Users\Sibam\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  """
C:\Users\Sibam\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  """
C:\Users\Sibam\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  """
C:\Users\Sibam\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning:

25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51


C:\Users\Sibam\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  """
C:\Users\Sibam\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  """
C:\Users\Sibam\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  """
C:\Users\Sibam\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning:

52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92


...

C:\Users\Sibam\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  """
C:\Users\Sibam\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  """
C:\Users\Sibam\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  """
C:\Users\Sibam\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning:

583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
601
602
603
604
605
606


C:\Users\Sibam\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  """
C:\Users\Sibam\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  """
C:\Users\Sibam\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  """
C:\Users\Sibam\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning:

607
608
609
610


C:\Users\Sibam\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  """


In [27]:
user_movie_pred

{1: [81132, 26701, 1907, 56152, 2987, 4719, 52462, 71999, 6902, 43932],
 2: [81132, 26236, 79132, 31367, 4956, 459, 49530, 91542, 8481, 31921],
 3: [31804, 36509, 43932, 3113, 90345, 7001, 2232, 184253, 32213, 3837],
 4: [81132, 26701, 56152, 2987, 1907, 81847, 4719, 32031, 52462, 43932],
 5: [81847, 2987, 56152, 1907, 81132, 595, 32031, 1064, 364, 2092],
 6: [81132, 81847, 85261, 32031, 56152, 1907, 26701, 52462, 364, 79132],
 7: [81132, 26701, 43932, 2987, 52462, 56152, 1907, 6902, 4719, 36509],
 8: [81847, 1907, 364, 56152, 595, 81132, 2987, 85261, 32031, 631],
 9: [81132, 2987, 56152, 52462, 32031, 36397, 81847, 84637, 4306, 92348],
 10: [56152, 81847, 1907, 36397, 85261, 32031, 2987, 4719, 51939, 108932],
 11: [81132, 79132, 43932, 60684, 36509, 7235, 27317, 27683, 26236, 8481],
 12: [71999, 4719, 26236, 4956, 164226, 117646, 587, 47404, 26764, 161594],
 13: [81132, 43932, 36509, 71999, 6902, 31804, 91542, 198, 27683, 8481],
 14: [81132, 79132, 60684, 36509, 43932, 8481, 27683, 19

In [28]:
max(list(movie_genre.dot(usergenre_matrix.loc[1])))

8.571582404153379

## Predicting 15 Movies Based on TimeStamp

In [29]:
ratings_75 = ratings[ratings['timestamp'] <= ratings.describe()['timestamp']['75%']]

In [30]:
ratings_25 = ratings[ratings['timestamp'] > ratings.describe()['timestamp']['75%']]

In [31]:
common = list(set(ratings_75['userId'].unique()).intersection(set(ratings_25['userId'].unique())))

In [32]:
rating75_groupby = pd.DataFrame(ratings_75.groupby(['userId', 'timestamp'])['movieId'].unique())

In [33]:
movies_temp = movies.copy()
movie_temp = movies_temp.set_index('movieId')

In [35]:
user_movie_pred2 = {}
for i in common:
    user_movie_pred2[i] = []

In [36]:
num = 5
for user in common:
    #print(user)
    count = 0
    while len(user_movie_pred2[user]) != 15:
        print(user, count)
        movie_id = (rating75_groupby.loc[user])[::-1].iloc[count]['movieId']
        for i in movie_id:
            array = (cosine_similarity(np.array(movie_genre.loc[i]).reshape(1, 19), movie_genre))[0].argsort()[-num:][::-1]
            user_movie_pred2[user].extend(list(movies.ix[array.tolist()].movieId))
            if num > 1:
                num -= 1
        user_movie_pred2[user] = list(set(user_movie_pred2[user]).difference(set(ratings_75[ratings_75['userId'] == user].movieId.tolist())))
        count += 1

522 0
522 1
522 2
522 3
522 4
522 5
522 6
522 7
522 8
522 9
522 10
522 11
15 0
15 1
15 2
15 3
15 4
15 5
15 6
15 7
15 8
15 9
15 10
15 11
15 12
15 13
15 14
15 15
21 0
21 1
21 2
21 3
21 4


C:\Users\Sibam\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Sibam\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Sibam\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-

21 5
21 6
21 7
21 8
21 9
21 10
21 11
21 12
21 13
21 14
21 15
21 16
21 17
21 18
21 19
21 20
21 21
21 22
537 0
537 1
537 2
537 3
537 4
537 5
537 6
537 7
537 8
537 9
537 10
537 11


C:\Users\Sibam\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Sibam\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Sibam\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-

537 12
537 13
537 14
537 15
537 16
537 17
537 18
537 19
537 20
537 21
282 0
282 1
282 2
282 3
282 4
282 5
282 6
282 7
282 8
282 9
282 10
282 11
282 12
282 13
282 14
282 15
282 16
282 17
282 18
282 19
282 20
414 0


...

C:\Users\Sibam\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Sibam\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Sibam\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-

 4
119 5
119 6
119 7
119 8
119 9
119 10
119 11
119 12
119 13
119 14
119 15
119 16
249 0
249 1
249 2
249 3
249 4
249 5
249 6
249 7
249 8
249 9
249 10
249 11
249

C:\Users\Sibam\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Sibam\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Sibam\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-

 12
249 13
249 14
249 15
249 16
249 17
249 18
249 19
249 20
249 21
249 22
509 0
509 1
509 2
509 3
509 4
509 5
509 6
509 7
509 8
509 9
509 10
509 11
509 12
509 13
509 14
509 15
509 16
509 17
509 18
509 19
509 20
509 21
509 22
509 23
509 24
509 25
509 26


C:\Users\Sibam\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Sibam\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Sibam\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-

In [37]:
user_movie_pred2

{522: [2688,
  85056,
  2467,
  26052,
  31364,
  193609,
  4139,
  55820,
  53996,
  140850,
  1620,
  101076,
  6934,
  6617,
  70015],
 15: [609,
  805,
  40581,
  48744,
  193609,
  153386,
  32460,
  5582,
  4015,
  1616,
  5423,
  49649,
  103219,
  1591,
  379],
 21: [2688,
  131714,
  45442,
  805,
  40581,
  96935,
  48744,
  193609,
  70015,
  133867,
  106542,
  135887,
  8950,
  4407,
  59103],
 537: [135137,
  55553,
  1732,
  193609,
  117867,
  99532,
  117133,
  49649,
  4338,
  59103,
  81910,
  1271,
  379,
  6686,
  70015],
 282: [26464,
  86721,
  7714,
  7299,
  168418,
  48744,
  193609,
  89745,
  4338,
  59103,
  1909,
  8950,
  2583,
  81910,
  70015],
 414: [7299,
  40581,
  4966,
  3879,
  48744,
  193609,
  99532,
  88140,
  135536,
  3696,
  111732,
  81910,
  2583,
  131610,
  1245],
 292: [135137,
  27426,
  45442,
  1732,
  119655,
  3879,
  58025,
  81834,
  8782,
  89745,
  2322,
  103253,
  2582,
  1591,
  131610],
 177: [609,
  7299,
  40581,
  48744

In [38]:
for user in user_movie_pred2:
    match = set(user_movie_pred2[user]).intersection(set(ratings_25[ratings_25['userId'] == 15].movieId.tolist()))
    print(user)
    print(list(match), len(match))

522
[] 0
15
[] 0
21
[] 0
537
[] 0
282
[89745] 1
414
[] 0
292
[89745] 1
177
[] 0
443
[89745] 1
318
[] 0
448
[] 0
193
[48780] 1
68
[] 0
326
[] 0
462
[89745] 1
103
[] 0
119
[] 0
249
[] 0
509
[] 0


# Predicting ratings given by Users

In [39]:
user_rating = pd.DataFrame(ratings_25.groupby(['userId', 'movieId'])['rating'].unique())

In [40]:
user_rating = pd.DataFrame(user_rating['rating'].apply(lambda x: user_rating['rating'][0][0]))

In [41]:
movies_temp = movies.copy()

In [42]:
for i in range(9742):
    movies_temp['genres'].iloc[i] = (movies_temp['genres'].iloc[i]).split('|')

C:\Users\Sibam\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [43]:
movies_temp = movies_temp.explode('genres')

In [44]:
result = pd.merge(movies_temp, ratings_75, left_on = 'movieId', right_on = 'movieId')

In [45]:
usergenre_count = pd.DataFrame(result.groupby(['userId', 'genres'])['genres'].count())

In [46]:
usergenre_count = usergenre_count.rename(columns={'genres': 'count'})

In [47]:
usergenre_count

count
userId genres          
1      Action        90
       Adventure     85
       Animation     29
       Children      42
       Comedy        83
...                 ...
609    Romance        5
       Sci-Fi         5
       Thriller      14
       War            4
       Western        1

[7971 rows x 1 columns]

In [48]:
genres = []
for i in movies.index:
    genres.extend(movies['genres'][i].split('|'))
genres = list(set(genres))
genres.remove('(no genres listed)')
users = list(ratings_75['userId'].unique())

In [49]:
usergenre_matrix_75 = pd.DataFrame(columns = genres, index = users)

In [50]:
x = result.groupby(['userId', 'genres']).mean()

In [51]:
x = x.drop(columns = ['movieId', 'timestamp'])

In [52]:
for i in genres:
    for j in users:
        try:
            usergenre_matrix_75[i][j] = x.loc[j, i][0]
        except:
            usergenre_matrix_75[i][j] = 0.0

In [53]:
usergenre_matrix_75

,Horror,Thriller,Musical,War,Animation,Drama,Action,Film-Noir,Children,Adventure,Fantasy,Western,Comedy,Mystery,Documentary,Crime,IMAX,Sci-Fi,Romance
1,3.47059,4.14545,4.68182,4.5,4.68966,4.52941,4.32222,5,4.54762,4.38824,4.29787,4.28571,4.27711,4.16667,0,4.35556,0,4.225,4.30769
3,4.6875,4.14286,0.5,0.5,0.5,0.75,3.57143,0,0.5,2.72727,3.375,0,1,5,0,0.5,0,4.2,0.5
4,4.25,3.55263,4,3.57143,4,3.48333,3.32,4,3.8,3.65517,3.68421,3.8,3.50962,3.47826,4,3.81481,3,2.83333,3.37931
5,3,3.55556,4.4,3.33333,4.33333,3.8,3.11111,0,4.11111,3.25,4.14286,3,3.46667,4,0,3.83333,3.66667,2.5,3.09091
6,3.26316,3.54412,4.16667,3.58333,4.07143,3.61429,3.60938,2.5,3.61702,3.89362,3.53846,3.81818,3.37008,3.73333,0,3.28571,4.66667,3.47619,3.61429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,2.86667,2.92,3.07692,2.77778,2.94231,3.11864,3.22464,0,3.20755,3.26415,3.15217,3.33333,3.25843,3.4375,3,3.04545,3.67857,3.28846,3.33051
606,3.34615,3.52513,3.72727,3.79231,3.71429,3.78797,3.17881,3.8125,3.44898,3.5034,3.59794,3.41176,3.56532,3.79121,3.8,3.65414,3.0625,3.55696,3.74085
607,4.11429,4.11475,3.6,4.16667,3.33333,4.0122,3.72222,0,3.42105,3.46667,3.57143,4,3.32727,4.64706,0,3.81481,5,3.25,3.51724
608,3.31959,3.53668,2.75758,3.57895,3.11818,3.4375,3.33032,3.75,2.46023,3.22099,3,2.63636,2.73662,3.55072,3,3.61301,4,3.29641,2.88679


In [54]:
movies_temp = movies.copy()

In [55]:
for i in range(9742):
    movies_temp['genres'].iloc[i] = (movies_temp['genres'].iloc[i]).split('|')

In [56]:
result25 = pd.merge(movies_temp, ratings_25, left_on = 'movieId', right_on = 'movieId')

In [57]:
result25['predicted_rating'] = 0

In [58]:
result25 = result25.set_index('userId')

In [59]:
result25 = result25.ix[common]

C:\Users\Sibam\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [60]:
result25 = result25.reset_index()

In [61]:
for user in result25['userId'].unique():
    for movie in result25[result25['userId'] == user].movieId:
        temp = 0
        value = 0
        for genre in list(result25[result25['userId'] == user][result25['movieId'] == movie].genres)[0]:
            try:
                value += usergenre_matrix_75[genre][user]*usergenre_count.loc[user, genre]['count']
                temp += usergenre_count.loc[user, genre]['count']
            except:
                pass
        try:
            value /= temp
        except ZeroDivisionError:
            value = 0
        result25.loc[(result25['userId'] == user) & (result25['movieId'] == movie), 'predicted_rating'] = value

C:\Users\Sibam\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


In [62]:
result25

,userId,movieId,title,genres,rating,timestamp,predicted_rating
0,522,1200,Aliens (1986),"[Action, Adventure, Horror, Sci-Fi]",4.5,1449731613,3.875000
1,522,1214,Alien (1979),"[Horror, Sci-Fi]",5.0,1449731619,3.980392
2,522,55820,No Country for Old Men (2007),"[Crime, Drama]",4.5,1449724924,3.900000
3,522,97304,Argo (2012),"[Drama, Thriller]",2.5,1449724955,3.893382
4,522,106100,Dallas Buyers Club (2013),[Drama],0.5,1449724936,3.833333
...,...,...,...,...,...,...,...
1838,509,129229,Northmen - A Viking Saga (2014),"[Action, Adventure]",2.0,1435997905,3.300000
1839,509,130073,Cinderella (2015),"[Children, Drama, Fantasy, Romance]",4.0,1435997996,3.375776
1840,509,133419,Pitch Perfect 2 (2015),[Comedy],5.0,1435997941,3.159091
1841,509,136838,Kiss me Kismet (2006),"[Comedy, Romance]",2.5,1435998776,3.199029
